# Tools and Routing

Previously, we showed how functions can be supplied to OpenAI models. 

They can be used to specify an output schema required for tasks (e.g., extraction, tagging) or for an API. 

Tools are more general and are a central concept in [LLM agents](https://python.langchain.com/docs/use_cases/more/agents/).

Often times, tools wrap APIs so that LLMs can easily access them. 

We can define tools (e.g., from functions) as shown below.

In [1]:
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]

In [2]:
!pip freeze | grep langchain

!pip freeze | grep openai

!pip freeze | grep pydantic

langchain==0.2.15
langchain-community==0.2.13
langchain-core==0.2.41
langchain-openai==0.1.23
langchain-text-splitters==0.2.4
langchain-openai==0.1.23
openai==1.47.0
openapi-schema-pydantic==1.2.4
pydantic==2.9.2
pydantic-settings==2.5.2
pydantic_core==2.23.4


In [3]:
from langchain.agents import tool

In [4]:
@tool
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

We can specify the tool input.

And, tools can be suppied as functions. 

**Note: May need to explain how this works.**

In [5]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

from langchain.tools.render import format_tool_to_openai_function
format_tool_to_openai_function(search)

/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6226/532786261.py:11: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(search)


{'name': 'search',
 'description': 'Search for the weather online.',
 'parameters': {'properties': {'query': {'description': 'Thing to search for',
    'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [6]:
search.run("sf")

'42f'

### Weather Tool


In [7]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [8]:
get_current_temperature.name

'get_current_temperature'

In [9]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [10]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [11]:
get_current_temperature({"latitude": 13, "longitude": 14})

/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6226/2143878976.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_current_temperature({"latitude": 13, "longitude": 14})
/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6226/4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 27.2°C'

In [12]:
get_current_temperature({"latitude": 40.3573, "longitude": -76.667})

/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6226/4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 20.6°C'

### Wikipedia Tool

In [13]:
#! pip install wikipedia               ###. This one does not work
#!python -m pip install wikipedia     ###. This one works!

In [14]:
#!pip freeze | grep wiki
import wikipedia

In [15]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [16]:
search_wikipedia.name

'search_wikipedia'

In [17]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [18]:
search_wikipedia.run({"query": "langchain?"})

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications that utilize large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.\n\nPage: DataStax\nSum

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [19]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
functions = [format_tool_to_openai_function(f) for f in [search_wikipedia, get_current_temperature]]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [20]:
model.invoke([HumanMessage(content="What is the weather in san francisco right now?")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 108, 'total_tokens': 133, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7be5cc35-304d-4ed1-b327-5b4bed863d83-0', usage_metadata={'input_tokens': 108, 'output_tokens': 25, 'total_tokens': 133})

In [21]:
model.invoke([HumanMessage(content="What is langchain")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 101, 'total_tokens': 117, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7c7ab4ce-5671-4e4c-bc04-7c43e6e47e4c-0', usage_metadata={'input_tokens': 101, 'output_tokens': 16, 'total_tokens': 117})

In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [23]:
chain.invoke({"input": "What is the weather in san francisco right now?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 116, 'total_tokens': 141, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-014c4f86-d47e-4a35-9c1c-9da694f9b756-0', usage_metadata={'input_tokens': 116, 'output_tokens': 25, 'total_tokens': 141})

In [24]:
chain.invoke({"input": "hi!"})

AIMessage(content='Well, hello there! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 107, 'total_tokens': 120, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9cf56c24-a519-4aa6-82d2-9ef149510be2-0', usage_metadata={'input_tokens': 107, 'output_tokens': 13, 'total_tokens': 120})

In [25]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [26]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [27]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [28]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [29]:
result.tool

'get_current_temperature'

In [30]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [31]:
get_current_temperature(result.tool_input)

/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6226/4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 15.8°C'

In [32]:
result = chain.invoke({"input": "hi!"})

In [33]:
type(result)

langchain_core.agents.AgentFinish

In [34]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [35]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [36]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [37]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

/var/folders/rm/ffmtyr9j47nccz_6s1wjrlzh0000gp/T/ipykernel_6226/4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [38]:
result

'The current temperature is 15.8°C'

In [39]:
result = chain.invoke({"input": "What is langchain?"})

In [40]:
result

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications that utilize large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.\n\nPage: DataStax\nSum

In [41]:
result = chain.invoke({"input": "Hi!"})

In [42]:
result

'Well, hello there! How can I assist you today?'